# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade Pillow

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding more one convolutional layer with more filters 

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 39s 157ms/step - loss: 0.6910 - accuracy: 0.5304 - val_loss: 0.6394 - val_accuracy: 0.6395
Epoch 2/25
250/250 [==============================] - 42s 166ms/step - loss: 0.6207 - accuracy: 0.6571 - val_loss: 0.5871 - val_accuracy: 0.6915
Epoch 3/25
250/250 [==============================] - 41s 164ms/step - loss: 0.5732 - accuracy: 0.7059 - val_loss: 0.5287 - val_accuracy: 0.7430
Epoch 4/25
250/250 [==============================] - 40s 159ms/step - loss: 0.5372 - accuracy: 0.7255 - val_loss: 0.4908 - val_accuracy: 0.7655
Epoch 5/25
250/250 [==============================] - 40s 162ms/step - loss: 0.5061 - accuracy: 0.7489 - val_loss: 0.4744 - val_accuracy: 0.7755
Epoch 6/25
250/250 [==============================] - 41s 163ms/step - loss: 0.4724 - accuracy: 0.7738 - val_loss: 0.4554 - val_accuracy: 0.7775
Epoch 7/25
250/250 [==============================] - 41s 163ms/step - loss: 0.4544 - accuracy: 0.7850 - val_loss: 0.4411 - val_ac

## Part 4 - Making a single prediction

In [17]:
import numpy as np
from tf.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

ModuleNotFoundError: No module named 'tf'

In [ ]:
print(prediction)

## Fim